# **CAPSTONE PROJECT | FINDING BEST AREAS IN CHENNAI**

In [1]:
# Installing Packages
!pip install geopy
!pip install folium
!pip install bs4

In [2]:
# Imporing Packages
import requests
from bs4 import BeautifulSoup

import pandas as pd

from geopy.geocoders import Nominatim

import folium
from IPython.display import display

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
#%matplotlib inline

from sklearn.cluster import KMeans


In [3]:
# Foursquare Credentials
CLIENT_ID = 'GBZNFATMC124BKW2WRQ20POA213WBBAAFNP3GJSIYFWC1KSV'
CLIENT_SECRET = 'EZT4UE440D3V3N0EWCL2RGOII2BRE2YDXQO4MPPWG4ZW0U51'
ACCESS_TOKEN = 'BX3DMHQLR401TPMMFSA1V0ZK3ML0QRZ1V0ZOV24GUMDKH0DY'
VERSION = '20180605'
LIMIT = 100

In [4]:
# Obtaining 'Chennai' location 
loc = 'Chennai'
geolocator = Nominatim(user_agent='foursquare_agent')
chn_loc = geolocator.geocode(loc)
chn_lat = chn_loc.latitude
chn_lon = chn_loc.longitude
print(chn_loc, chn_lat, chn_lon)

Chennai, Chennai District, Tamil Nadu, India 13.0836939 80.270186


In [7]:
# Scrapping List of names of Areas of Chennai 
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Chennai'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')
area_list = list()
table = soup.findAll('li')
for li in table:
    area_list.append(li.text)
area_list = area_list[13:192]

In [6]:
# Removing redundant areas
invalid = ['Old Washermenpet','New Washermenpet','Manali New Town','Naravarikuppam','Mannadi',
           'Central','Surapet','Manali','George Town','M.K.B. Nagar','Selavoyal','Defence Colony',
           'Manjambakkam','Ponniammanmedu','Pattravakkam','T.V.K. Nagar','Erukanchery','Park Town',
           'Kallikuppam','Karambakkam','Thirunindravur','Thirumullaivayal','Gummidipoondi','Alandur',
           'Nerkundrum','M.G.R. Garden','Mowlivakkam','Ramapuram','CMDA Colony','Ennore','Pallavaram',
           'M.G.R. Nagar','Mangadu','Pudur','Thirumangalam','Gopalapuram','Hastinapuram','Tiruvallur',
           'Nagalkeni','Kolappakkam','Athipattu','Thirunindravur','Kadambathur','Kanaythur','Ponneri',
           'Pudupet','Chetpet','Ayappakkam','Mambakkam','K.K. Nagar','Pattalam','Varadharajapuram',
           'Lakshmipuram','Singaperumalkoil','Broadway','Kovalam','Kadaperi','Muthukadu','Moolakadai',
           'Sholinganallur','Periamet','Jamalia, Chennai','Purasawalkam','Pazhaverkadu']
for area in invalid:
    area_list.remove(area)

In [7]:
# Obtaining the coordinates
temp_list = list()
for area in area_list:
    cell={}
    cell["Area"] = area
    area = Nominatim(user_agent='foursquare_agent').geocode(area)
    cell["Latitude"] = area.latitude
    cell['Longitude'] = area.longitude
    temp_list.append(cell)

# Adding the left out Areas
area_list2 = ['Chetpet','Pudupet','Kanathur','Tiruninravur','Manali', 'Central','KK Nagar',
              'George Town','MKB Nagar','Erukkancheri','Ayanavaram','MGR Nagar','Nungambakkam',
              'Defence Colony','MGR Garden','Moulivakkam','Ramapuram','MMDA Colony','Attipattu',
              'Mangadu','Thirumangalam','Gopalapuram','Kolapakkam','Washermenpet','Choolaimedu',
              'Singaperumal koil','Alwarpet']

for area in area_list2:
    cell={}
    cell["Area"] = area
    area = Nominatim(user_agent='foursquare_agent').geocode(f'{area}, Chennai')
    cell["Latitude"] = area.latitude
    cell['Longitude'] = area.longitude
    temp_list.append(cell)
    
print('Coordinates obtained successfully')

Coordinates obtained successfully


In [8]:
df = pd.DataFrame(temp_list)

# Inserting corrected coordinates
temp = pd.DataFrame([{'Area': 'Broadway' ,"Latitude": '13.05466',"Longitude": '80.17109' },
                     {"Area": 'Poonamallee' ,"Latitude": '13.047185',"Longitude": '80.095072'},
                     {"Area": 'Thirumullaivayal' ,"Latitude": '13.132915',"Longitude": '80.134651'},
                     {"Area": 'Mambakkam' ,"Latitude": '12.838601',"Longitude": '80.169790'},
                     {"Area": 'Hasthinapuram' ,"Latitude": '12.938719',"Longitude": '80.147781'},
                     {"Area": 'Lakshmipuram' ,"Latitude": '13.136087',"Longitude": '80.200403'},
                     {"Area": 'Sholinganallur' ,"Latitude": '12.900875',"Longitude": '80.227818'}])

df = df.append(temp, ignore_index=True)
df.head()

,Area,Latitude,Longitude
0,Red Hills,13.192279,80.183766
1,Royapuram,13.10429,80.293613
2,Korukkupet,13.121534,80.283791
3,Vyasarpadi,13.116973,80.257124
4,Tondiarpet,13.127767,80.289585
...,...,...,...
144,Thirumullaivayal,13.132915,80.134651
145,Mambakkam,12.838601,80.169790
146,Hasthinapuram,12.938719,80.147781
147,Lakshmipuram,13.136087,80.200403


In [45]:
chennai_area_location = pd.read_csv('df_loc.csv')

In [46]:
chennai_area_location.head()

,Area,Latitude,Longitude
0,Adambakkam,12.982221,80.209121
1,Adyar,13.006450,80.257779
2,Alapakkam,13.049901,80.165435
3,Alwarpet,13.033860,80.254549
4,Alwarthirunagar,13.047486,80.183669


In [47]:
# Displaying Areas of Chennai
map_clusters = folium.Map(location=[chn_lat, chn_lon-0.1], zoom_start=11)

for lat, lon, name in zip(chennai_area_location['Latitude'], chennai_area_location['Longitude'], chennai_area_location['Area']):
    label = folium.Popup(name)
    folium.CircleMarker([lat,lon], radius=5, popup=label,
                      color='red', fill=True).add_to(map_clusters)

map_clusters

In [10]:
# Exploring each Area
venue_list = list()
radius = 1000

for area, lat, lon in zip(chennai_area_location['Area'], chennai_area_location['Latitude'], chennai_area_location['Longitude']):
    # Searching Area for specific venue category
    search_list = ['school','hospital','market','Restaurant','hotel','theatre']
    for search_query in search_list:
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, lat, lon, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        try:
            venues = requests.get(url).json()['response']['venues']
            if venues:
                venue_list.append([(area, v['categories'][0]['name']) for v in venues])
        except:
            pass

print('Venues loaded successfully')

Venues loaded successfully


In [11]:
chennai_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
chennai_venues.columns = ['Area', 'Venue Category']
chennai_venues

,Area,Venue Category
0,Red Hills,Student Center
1,Red Hills,High School
2,Red Hills,High School
3,Red Hills,Department Store
4,Red Hills,Fishing Store
...,...,...
1054,Thiruvanmiyur,Supermarket
1055,Thiruvanmiyur,Farmers Market
1056,Thiruvanmiyur,Supermarket
1057,Thiruvanmiyur,Fish Market


{'file_name': 'df_venues2.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'pythondatascienceproject-donotdelete-pr-f9x0jcaap6asso',
 'asset_id': '0638ceb4-832b-4004-a441-24e4041d3869'}

In [135]:
chennai_venues = pd.read_csv('chennai_venues.csv')
chennai_venues

,Area,Venue Category
0,Red Hills,Student Center
1,Red Hills,High School
2,Red Hills,High School
3,Red Hills,Department Store
4,Red Hills,Fishing Store
...,...,...
1946,Sholinganallur,Multicuisine Indian Restaurant
1947,Sholinganallur,Restaurant
1948,Sholinganallur,Hotel
1949,Sholinganallur,Coffee Shop


In [136]:
# one hot encoding
chennai_onehot = pd.get_dummies(chennai_venues[['Venue Category']], prefix="", prefix_sep="")
chennai_onehot['Area'] = chennai_venues['Area']

fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]

chennai_grouped = chennai_onehot.groupby('Area').sum().reset_index()
chennai_grouped.head()

,Area,Adult Education Center,Advertising Agency,Afghan Restaurant,African Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,...,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Trade School,Travel Agency,University,Vegetarian / Vegan Restaurant,Veterinarian,Well
0,Adambakkam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adyar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Alapakkam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alwarpet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alwarthirunagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [137]:
# Function to combine venue columns together
def combine_columns (df, column_name, findby=None, columns=None):
    all_venue_category = list(chennai_grouped.columns)

    if columns==None:
        columns = list()

    if findby:
        for i in all_venue_category:
            if i.find(findby) > 0:
                columns.append(i)

    temp = list()
    for i, j in enumerate(df['Area']):
        sum = 0
        for k in columns:
            sum += df.loc[i, k]
        temp.append(sum)

    tempdf = pd.DataFrame({column_name: [i for i in temp]})
    df.drop(axis=0, columns=[i for i in columns], inplace=True)
    df = pd.concat([df, tempdf], axis=1)
    return df

In [138]:
chennai_grouped = combine_columns(chennai_grouped, column_name='Restaurant', findby='Restaurant', columns=['Restaurant'])
chennai_grouped = combine_columns(chennai_grouped, column_name='School', findby='School', columns=['School'])
chennai_grouped = combine_columns(chennai_grouped, column_name='Bus Stop', columns=['Bus Line', 'Bus Station', 'Bus Stop'])
chennai_grouped = combine_columns(chennai_grouped, column_name='Shopping Mall', columns=['Shopping Mall', 'Shopping Plaza'])
chennai_grouped = combine_columns(chennai_grouped, column_name='Gym', columns=['Gym', 'Gym / Fitness Center'])
chennai_grouped = combine_columns(chennai_grouped, column_name='Theater', columns=['Movie Theater','Multiplex','Theater'])
chennai_grouped.head()

,Area,Adult Education Center,Advertising Agency,Antique Shop,Aquarium,Arcade,Automotive Shop,BBQ Joint,Bakery,Bar,...,Travel Agency,University,Veterinarian,Well,Restaurant,School,Bus Stop,Shopping Mall,Gym,Theater
0,Adambakkam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,0
1,Adyar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,11,0,1,0,0,0
2,Alapakkam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alwarpet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Alwarthirunagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,17,0,0,0,0,0


In [139]:
retain_venues = ['Area','School','Hospital','Medical Center','Grocery Store','Convenience Store','Department Store','Hardware Store','Supermarket','Fruit & Vegetable Store','Bakery','Restaurant','Hotel','Theater','Bus Stop','Shopping Mall','Gym','Park','Playground','Indoor Play Area']
chennai_venues = chennai_grouped[retain_venues]
chennai_venues

,Area,School,Hospital,Medical Center,Grocery Store,Convenience Store,Department Store,Hardware Store,Supermarket,Fruit & Vegetable Store,Bakery,Restaurant,Hotel,Theater,Bus Stop,Shopping Mall,Gym,Park,Playground,Indoor Play Area
0,Adambakkam,4,4,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adyar,0,7,0,0,1,0,0,0,0,0,11,0,0,1,0,0,0,0,0
2,Alapakkam,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alwarpet,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,Alwarthirunagar,0,0,0,0,1,1,0,0,0,0,17,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Virugambakkam,0,3,0,0,0,1,0,0,0,0,6,0,1,0,0,0,0,0,0
127,Vyasarpadi,2,3,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
128,Washermenpet,1,10,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0
129,West Mambalam,10,8,0,1,0,1,0,0,0,0,8,0,0,1,0,0,0,1,0


In [140]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 3
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

# create a new dataframe
chennai_venues_sorted = pd.DataFrame(columns=columns)
chennai_venues_sorted['Area'] = chennai_venues['Area']

for i in np.arange(chennai_grouped.shape[0]):
    chennai_venues_sorted.iloc[i, 1:] = return_most_common_venues(chennai_venues.iloc[i, :], num_top_venues)

chennai_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Adambakkam,School,Hospital,Department Store
1,Adyar,Restaurant,Hospital,Bus Stop
2,Alapakkam,Hospital,Indoor Play Area,Fruit & Vegetable Store
3,Alwarpet,Restaurant,Indoor Play Area,Fruit & Vegetable Store
4,Alwarthirunagar,Restaurant,Convenience Store,Department Store


In [141]:
chennai_venues = pd.merge(chennai_venues, chennai_area_location, on='Area')
fixed_columns = [chennai_venues.columns[-2], chennai_venues.columns[-1]] + list(chennai_venues.columns[:-2])
chennai_area = chennai_venues[fixed_columns]
chennai_area

,Latitude,Longitude,Area,School,Hospital,Medical Center,Grocery Store,Convenience Store,Department Store,Hardware Store,...,Bakery,Restaurant,Hotel,Theater,Bus Stop,Shopping Mall,Gym,Park,Playground,Indoor Play Area
0,12.982221,80.209121,Adambakkam,4,4,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,13.006450,80.257779,Adyar,0,7,0,0,1,0,0,...,0,11,0,0,1,0,0,0,0,0
2,13.049901,80.165435,Alapakkam,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13.033860,80.254549,Alwarpet,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,13.047486,80.183669,Alwarthirunagar,0,0,0,0,1,1,0,...,0,17,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,13.047543,80.192218,Virugambakkam,0,3,0,0,0,1,0,...,0,6,0,1,0,0,0,0,0,0
127,13.116973,80.257124,Vyasarpadi,2,3,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
128,13.111656,80.284745,Washermenpet,1,10,0,0,0,0,0,...,0,1,0,2,1,0,0,0,0,0
129,13.042732,80.227078,West Mambalam,10,8,0,1,0,1,0,...,0,8,0,0,1,0,0,0,1,0


In [142]:
# Clustering the Areas

kclusters = 5
chennai_area_clustering = chennai_area.drop(['Area'], axis=1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(chennai_area_clustering)

In [143]:
chennai_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
chennai_venues_sorted

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Adambakkam,1,School,Hospital,Department Store
1,Adyar,4,Restaurant,Hospital,Bus Stop
2,Alapakkam,1,Hospital,Indoor Play Area,Fruit & Vegetable Store
3,Alwarpet,1,Restaurant,Indoor Play Area,Fruit & Vegetable Store
4,Alwarthirunagar,4,Restaurant,Convenience Store,Department Store
...,...,...,...,...,...
126,Virugambakkam,2,Restaurant,Hospital,Theater
127,Vyasarpadi,1,Hospital,School,Fruit & Vegetable Store
128,Washermenpet,2,Hospital,Theater,School
129,West Mambalam,0,School,Hospital,Restaurant


In [144]:
# create map
map_clusters = folium.Map(location=[chn_lat, chn_lon-0.1], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_area_clustered['Latitude'], chennai_area_clustered['Longitude'], chennai_area_clustered['Area'], map(int, chennai_area_clustered['Cluster Labels'])):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [145]:
# Cluster 1
chennai_venues_sorted.loc[chennai_venues_sorted['Cluster Labels'] == 0]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
7,Anna Nagar,0,Restaurant,Hospital,School
18,Chetpet,0,Hospital,Restaurant,Hotel
70,Nanganallur,0,Hospital,School,Restaurant
103,T. Nagar,0,Restaurant,Hospital,School
119,Vadapalani,0,Restaurant,Hospital,School
129,West Mambalam,0,School,Hospital,Restaurant


In [146]:
# Cluster 2
chennai_venues_sorted.loc[chennai_venues_sorted['Cluster Labels'] == 1]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Adambakkam,1,School,Hospital,Department Store
2,Alapakkam,1,Hospital,Indoor Play Area,Fruit & Vegetable Store
3,Alwarpet,1,Restaurant,Indoor Play Area,Fruit & Vegetable Store
12,Ayapakkam,1,Supermarket,Indoor Play Area,Fruit & Vegetable Store
13,Basin Bridge,1,Theater,Indoor Play Area,Fruit & Vegetable Store
14,Besant Nagar,1,School,Hospital,Supermarket
15,Broadway,1,Hospital,Indoor Play Area,Fruit & Vegetable Store
17,Chembarambakkam,1,Restaurant,Indoor Play Area,Fruit & Vegetable Store
24,Erukkancheri,1,School,Fruit & Vegetable Store,Medical Center
26,Gerugambakkam,1,Indoor Play Area,Fruit & Vegetable Store,Hospital


In [147]:
# Cluster 3
chennai_venues_sorted.loc[chennai_venues_sorted['Cluster Labels'] == 2]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
5,Ambattur,2,Restaurant,Hospital,Theater
6,Aminjikarai,2,School,Restaurant,Department Store
10,Avadi,2,Restaurant,School,Hospital
11,Ayanavaram,2,Hospital,Restaurant,School
21,Chromepet,2,Restaurant,Hospital,Supermarket
22,Defence Colony,2,Restaurant,School,Hospital
23,Egmore,2,Hospital,Theater,Fruit & Vegetable Store
28,Guduvanchery,2,Restaurant,Indoor Play Area,Fruit & Vegetable Store
31,ICF Colony,2,Hospital,Department Store,Restaurant
34,Iyyapanthangal,2,Restaurant,Hospital,Hotel


In [148]:
# Cluster 4
chennai_venues_sorted.loc[chennai_venues_sorted['Cluster Labels'] == 3]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
27,Gopalapuram,3,Restaurant,Department Store,Grocery Store
81,Parry's Corner,3,Restaurant,School,Hospital
93,Ramapuram,3,Restaurant,School,Hotel
96,Saidapet,3,Restaurant,Hotel,Bus Stop
106,Teynampet,3,Restaurant,Department Store,Indoor Play Area


In [149]:
# Cluster 5
chennai_venues_sorted.loc[chennai_venues_sorted['Cluster Labels'] == 4]

,Area,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Adyar,4,Restaurant,Hospital,Bus Stop
4,Alwarthirunagar,4,Restaurant,Convenience Store,Department Store
8,Arumbakkam,4,Restaurant,School,Department Store
9,Ashok Nagar,4,Restaurant,School,Department Store
16,Central,4,Restaurant,School,Hospital
19,Choolai,4,Restaurant,Hotel,Hospital
20,Choolaimedu,4,Restaurant,School,Department Store
25,George Town,4,Restaurant,School,Theater
35,KK Nagar,4,School,Restaurant,Theater
47,Kosapet,4,Restaurant,Hospital,Hotel
